In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.models.doc2vec import TaggedDocument
import multiprocessing
from sklearn import utils
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings(action='ignore')
SEED = 2000

In [2]:
data = pd.read_csv("../data/clean_tweet.csv", index_col=0)
data.head()

,text,target
0,awww that bummer you shoulda got david carr of...,0
1,is upset that he can not update his facebook b...,0
2,dived many times for the ball managed to save ...,0
3,my whole body feels itchy and like its on fire,0
4,no it not behaving at all mad why am here beca...,0


In [3]:
x = data.text
y = data.target

In [4]:
x_train, x_validation_test, y_train, y_validation_test = train_test_split(x,y, test_size=0.02, random_state=SEED)

In [5]:
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_test, y_validation_test, test_size=0.5, random_state=SEED)

In [6]:
print(f"Train set has total {len(x_train)} with {len(x_train[y_train==0])*100/(len(x_train)*1.0)}% negative and {len(x_train[y_train==4])*100/(len(x_train)*1.0)}% positive")
print(f"Validation set has total {len(x_validation)} with {len(x_validation[y_validation==0])*100/(len(x_validation)*1.0)}% negative and {len(x_validation[y_validation==4])*100/(len(x_validation)*1.0)}% positive")
print(f"Test set has total {len(x_test)} with {len(x_test[y_test==0])*100/(len(x_test)*1.0)}% negative and {len(x_test[y_test==4])*100/(len(x_test)*1.0)}% positive")

Train set has total 1564120 with 50.020139119760636% negative and 0.0% positive
Validation set has total 15960 with 49.454887218045116% negative and 0.0% positive
Test set has total 15961 with 49.67733851262452% negative and 0.0% positive


In [10]:
tokenized_train = [t.split() for t in x_train]
phrases = Phrases(tokenized_train)
bigram = Phraser(phrases)

In [20]:
def labelize_tweets_bg(tweets, label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(TaggedDocument(bigram[t.split()], [prefix + '_%s' % i]))
    return result

In [21]:
all_data = pd.concat([x_train, x_validation, x_test])
all_data_w2v_bg = labelize_tweets_bg(all_data, 'doc')

### Bigrams and Trigrams using Phrases and Phraser

Do the same process as we did with the unigrams, we can repeat the process by creatimg trigrams as well. The improvement in result is not significant.

In [ ]:
cores = multiprocessing.cpu_count()
model_bg_dbow = Doc2Vec(dm=0, vector_size=100, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_bg_dbow.build_vocab([x for x in tqdm(all_data_w2v_bg)])

for epoch in range(30):
    model_bg_dbow.train(utils.shuffle([x for x in tqdm(all_data_w2v_bg)]), total_examples=len(all_data_w2v_bg), epochs=1)
    model_bg_dbow.alpha -= 0.002
    model_bg_dbow.min_alpha = model_bg_dbow.alpha
    
train_vecs_dbow_bg = get_vectors(model_bg_dbow, x_train, 100)
validation_vecs_dbow_bg = get_vectors(model_bg_dbow, x_validation, 100)

clf = LogisticRegression()
clf.fit(train_vecs_dbow_bg, y_train)
clf.score(validation_vecs_dbow_bg, y_validation)